# 強化学習の学習
強化学習を自習するためのノートブック

## 環境の定義
まず，強化学習エージェントを動かすための環境を定義する．

ここでは，強化学習のテキスト等でよく扱われる三目並べ（tic-tac-toe）と多腕バンディット（multi-armed bandit）問題を環境として用いる．

### 必要なパッケージのインポート

In [1]:
import gym
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### env1: tic-tac-toe

In [6]:
import gym_tic_tac_toe
env1 = gym.make("tic_tac_toe-v0")
  
env1.laction_space

[2017-09-12 17:42:44,550] Making new env: tic_tac_toe-v0


Discrete(9)

### env2: multi-armed bandit

In [4]:
import gym_bandits
env2 = gym.make("BanditTenArmedGaussian-v0")

[2017-09-12 17:41:59,795] Making new env: BanditTenArmedGaussian-v0
/Users/eichi/.pyenv/versions/anaconda3-4.1.1/lib/python3.5/site-packages/numpy/core/numeric.py:301: FutureWarning: in the future, full(10, 1) will return an array of dtype('int64')
  format(shape, fill_value, array(fill_value).dtype), FutureWarning)


In [5]:
env2

<TimeLimit<BanditTenArmedGaussian<BanditTenArmedGaussian-v0>>>